# ResNet on CIFAR10

Residual Network is a model proposed the first time by [K.  He et al in 2015](https://arxiv.org/pdf/1512.03385.pdf), and a reviewed version in 2016 once again by [K. He et al](https://arxiv.org/pdf/1603.05027.pdf).  The idea of the ResNet, is similar to the GoogLeNet, the authors consider the idea of micro-architecture to build the macro-architecture.

The micro-architecture is called of residual module. This module performs a certain number of convolution operation, considering different kernel sizes, to reduce the volume and avoid maxpooling operation. At the end of the residual module, we add the shortcut, that’s the input vector  passed at the top of the residual module, this shortcut enables the module to create a map of features. Each residual module has an associated number of filters. Using micro-architecture, the model is enable to increase the depth of the network, without increasing the running time. The head of the model is a softmax operation, to classify the images. 

In the compvis module, there’s two version of ResNet model, a shallow and deep version. The shallow version will be used  in this example, using the CIFAR10 dataset. The family of ResNet is composed by several model, each model is defined by the total amount of residual module, for example, ResNet50 has 50 residual module.  We can define how many residual layers we want.

## Importing Libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelBinarizer
from compvis.nn.cnns import ResNet
from compvis.callbacks import TrainingMonitor
from compvis.nn.lr import LRFunc
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
import numpy as np
import sys
import os
sys.setrecursionlimit(5000)

## Loading and preprocessing dataset

In [2]:
# Loading and splitting the dataset
((X_train, y_train), (X_test, y_test)) = cifar10.load_data()
X_train = X_train.astype('float32')
X_test = X_test.astype("float32")

In [3]:
# Normalizing with mean substraction
mean = np.mean(X_train, axis = 0)
X_train -= mean
X_test -= mean

In [4]:
# Converting the labels into numercial vectors
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.fit_transform(y_test)

## Building the model

To build the model, we consider the class ResNet. Before define and training the model, we need to set some regularization and, define the learning rate scheduler that will be used during the training. 

**Learning rate polynomial function decay**

During the training, the learning rate will be dropped down, at each step, we consider a polynomial function to decrease the learning rate.

We consider the class LRFunc and your attribute, poly_decay. When using the poly_decay, we must pass some arguments as initial learning rate, total number of epochs and the power of the polynomial function, in our training we consider a linear function.

In [5]:
init_lr = 1e-1
epochs = 100
deg = 1

In [6]:
lrs = LRFunc(l_r = init_lr, epochs = epochs, degree=deg) # defining the LRFunc class

**Image augmentation**

In [7]:
# Defining the Data augumentation to avoid the overfit
aug = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1,
                         horizontal_flip=True, fill_mode="nearest")

**Defining Callbacks**

Using the callback class from TensorFlow, we can consider the Training Monitor (print out of the learning curves) and the LearningRateScheduler, to reduce the learning rate using the poly_decay attribute.

In [8]:
# Path to store the history of learning curves and their graphics
figPath = os.path.sep.join(["/output", "{}.png".format(os.getpid())])
jsonPath = os.path.sep.join(["/output", "{}.json".format(os.getpid())])

In [9]:
# Defining the callbacks
callbacks = [TrainingMonitor(figPath, jsonPath=jsonPath),
             LearningRateScheduler(lrs.poly_decay)]#we consider the attribute poly_decay

**Building the model**

Using the class ResNet, we have the attribute build, that requires some arguments as, width, height, depth, number of class, stage, filters and regularization value.

The argument stage is a tuple of int values. These values correspond to the total of inception modules, it means, if we have $9$ as value, the model will stack $9$ residual modules, building a block of residual. The argument filter is another tuple of int values. The values correspond to the total number of filters in the convolution process. The first value in the tuple does not have correspondence with the residual modules, this value will be applied on the first convolution operation before starting the residual blocks. 

For example, when we pass $9$ as the first value of the stage tuple, the convolutional process will have $64$ filter, that is the second value in the filters tuple. When we pass 9 the second value in the stage tuple, the correspondent in the filters tuple will be $128$ and so on.

In [10]:
stage = (9, 9, 9)
filters = (64, 64, 128, 256)
nclass = 10

In [11]:
# Constructing the model and the optimizer
opt = SGD(lr=init_lr, momentum=0.9)
model = ResNet.build(32, 32, 3, nclass, stage, filters, reg=0.0005)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

**Training the model**

We train the model over $100$ epochs and batch size of $128$.

In [12]:
model.fit(aug.flow(X_train, y_train, batch_size=128), validation_data=(X_test, y_test),
          steps_per_epoch=len(X_train) // 128, epochs=epochs, 
          callbacks=callbacks, verbose=1)

  ...
    to  
  ['...']
Train for 390 steps, validate on 10000 samples


The parameters for the Polynomial Decay function:
- Initial learning rate: 0.100000 
- Epochs: 100
- Degree: 1.

Epoch 1/100
390/390 [==============================] - 92s 236ms/step - loss: 2.1904 - accuracy: 0.4121 - val_loss: 2.1631 - val_accuracy: 0.4579
Epoch 2/100
390/390 [==============================] - 84s 215ms/step - loss: 1.6437 - accuracy: 0.5933 - val_loss: 1.6548 - val_accuracy: 0.5995
Epoch 3/100
390/390 [==============================] - 88s 226ms/step - loss: 1.3537 - accuracy: 0.6866 - val_loss: 1.5536 - val_accuracy: 0.6284
Epoch 4/100
390/390 [==============================] - 88s 224ms/step - loss: 1.1716 - accuracy: 0.7392 - val_loss: 1.2205 - val_accuracy: 0.7249
Epoch 5/100
390/390 [==============================] - 85s 219ms/step - loss: 1.0645 - accuracy: 0.7672 - val_loss: 1.1604 - val_accuracy: 0.7395
Epoch 6/100
390/390 [==============================] - 87s 222ms/step - loss: 0.983

Epoch 54/100
390/390 [==============================] - 83s 213ms/step - loss: 0.4251 - accuracy: 0.9371 - val_loss: 0.6067 - val_accuracy: 0.8809
Epoch 55/100
390/390 [==============================] - 83s 213ms/step - loss: 0.4200 - accuracy: 0.9395 - val_loss: 0.6312 - val_accuracy: 0.8807
Epoch 56/100
390/390 [==============================] - 83s 212ms/step - loss: 0.4085 - accuracy: 0.9426 - val_loss: 0.6537 - val_accuracy: 0.8763
Epoch 57/100
390/390 [==============================] - 83s 213ms/step - loss: 0.4063 - accuracy: 0.9428 - val_loss: 0.6129 - val_accuracy: 0.8838
Epoch 58/100
390/390 [==============================] - 83s 213ms/step - loss: 0.3968 - accuracy: 0.9451 - val_loss: 0.6075 - val_accuracy: 0.8852
Epoch 59/100
390/390 [==============================] - 83s 213ms/step - loss: 0.3935 - accuracy: 0.9451 - val_loss: 0.5447 - val_accuracy: 0.8983
Epoch 60/100
390/390 [==============================] - 83s 213ms/step - loss: 0.3925 - accuracy: 0.9470 - val_loss: 0

In [13]:
model.save('output/model_lr1.hdf5')

## Conclusions

The results after the training are interesting, we've obtained $0.9297$ as accuracy on the validation set. On the other hand, if we look with attention, the over-fit persists on the CIFAR10 dataset, maybe to obtain a better results, we must use transfer learning. Until now, in this training repository, this is the best result on the validation set, that's a good result for a network trained from scratch. The graphic with the results is found here. 